# Marine mammal and turtle movement areas
This code creates areas to use as a mask in 2023 Southeast Conservation Blueprint marine corridors analysis

Code written by Rua Mordecai 

In [1]:
import os, string
import arcpy

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutWorkspace = "D:/Blueprint/2023/corridors/MarineMammalMovement/working"

In [3]:
# define final output name
maskFileName = "MarineMammalAndTurtleMovementAreas.tif"

In [4]:
# define rasters used for cell size, extent, and snapping
MarineRaster = r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"

In [5]:
# species data folder
speciesFolder = r"D:\Blueprint\2023\corridors\MarineMammalMovement\speciesData"

In [6]:
# pilot whale data. Unlike the other species, this only has one yearly layer so we treat it differently
AtlanticPilot = r"D:\Blueprint\2023\corridors\MarineMammalMovement\speciesData\PIWH_v7_density.img"

In [7]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

### Combine marine mammal monthly abundance

In [24]:
# Set the workspace where the marine mammal monthly data are stored
arcpy.env.workspace = speciesFolder

In [25]:
# find all the species rasters
speciesFileList = arcpy.ListRasters()

In [26]:
# make empty lists for the individual species
pilotList = []
riceList = []
loggerheadList = []
kempsList = []
rightList = []
humpbackList = []

In [27]:
# make species specific file lists
for i in speciesFileList:
    if i[0:5] == "Pilot":
        pilotList.append(i)
    elif i[0:5] == "Rices":
        riceList.append(i)    
    elif i[0:5] == "logge":
        loggerheadList.append(i)
    elif i[0:5] == "kemps":
        kempsList.append(i)
    elif i[0:5] == "NARW_":
        rightList.append(i)
    elif i[0:5] == "HUWH_":
        humpbackList.append(i)    
    else:
        print("error in species list")

In [28]:
# make a consecutive list of letters for the 12 months of data for each raster
formulaLetters = list(string.ascii_lowercase)[:12]

In [29]:
# make a formula using those letters
spFormula = ''
for i in formulaLetters:
    spFormula += i + " + "
# remove the last + from the formula
spFormula = spFormula[:-3]    

In [34]:
#Sum pilot whale rasters
output_raster = arcpy.sa.RasterCalculator(pilotList,formulaLetters,spFormula); output_raster.save(OutWorkspace + "/" + "pilotSum.tif")

In [35]:
#Sum rice whale rasters
output_raster = arcpy.sa.RasterCalculator(riceList,formulaLetters,spFormula); output_raster.save(OutWorkspace + "/" + "riceSum.tif")

In [13]:
#Sum gulf loggerhead sea turtle rasters
output_raster = arcpy.sa.RasterCalculator(loggerheadList,formulaLetters,spFormula); output_raster.save(OutWorkspace + "/" + "loggerheadSum.tif")

In [14]:
#Sum gulf kemps sea turtle rasters
output_raster = arcpy.sa.RasterCalculator(kempsList,formulaLetters,spFormula); output_raster.save(OutWorkspace + "/" + "kempsSum.tif")

In [22]:
#Sum Atlantic Right whale rasters
output_raster = arcpy.sa.RasterCalculator(rightList,formulaLetters,spFormula); output_raster.save(OutWorkspace + "/" + "rightSum.tif")

In [23]:
#Sum Atlantic humpback whale rasters
output_raster = arcpy.sa.RasterCalculator(humpbackList,formulaLetters,spFormula); output_raster.save(OutWorkspace + "/" + "humpbackSum.tif")

## Apply species specific cutoffs

In [8]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [8]:
# reclassify species using the top 70% based on a 10 class quantile classification

In [9]:
out_raster = arcpy.sa.Reclassify("riceSum.tif", "Value", "0 0.097 0;0.097 8.135 1", "DATA"); out_raster.save("riceBin.tif")

In [31]:
# for this species only include the top 60% to make more cohesive and selective corridors
out_raster = arcpy.sa.Reclassify("pilotSum.tif", "Value", "0 1.183 0;1.183 21.543 1", "DATA"); out_raster.save("pilotBin.tif")

In [32]:
# for this species only include the top 50% to make more cohesive and selective corridors
out_raster = arcpy.sa.Reclassify("loggerheadSum.tif", "Value", "0 149.581 0;149.581 2007.517 1", "DATA"); out_raster.save("loggerheadBin.tif")

In [13]:
out_raster = arcpy.sa.Reclassify("kempsSum.tif", "Value", "0 39.072 0;39.072 2490.783 1", "DATA"); out_raster.save("kempsBin.tif")

In [12]:
out_raster = arcpy.sa.Reclassify("rightSum.tif", "Value", "0 0.198 0;0.198 25.137 1", "DATA"); out_raster.save("rightBin.tif")

In [10]:
# This one has a defined threshold of 0.4 to created a complete corridor for the species and match the 2022 approach
out_raster = arcpy.sa.Reclassify("humpbackSum.tif", "Value", "0 0.04 0;0.04 45.723 1", "DATA"); out_raster.save("humpbackBin.tif")

In [11]:
out_raster = arcpy.sa.Reclassify(AtlanticPilot, "Value", "0 0.7889 0;0.7889 100.6 1", "DATA"); out_raster.save("atlanticPilotBin.tif")

### Combine species corridors

In [9]:
# combine gulf species corridors (loggerhead, kemps, rices, and pilot) and align with Marine extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.CellStatistics("kempsBin.tif;loggerheadBin.tif;pilotBin.tif;riceBin.tif", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("GulfCorridor.tif")

In [10]:
# combine atlantic species corridors (right, pilot, and humpback) and align with Marine extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.CellStatistics("rightBin.tif;humpbackBin.tif;atlanticPilotBin.tif", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("AtlanticCorridor.tif")

In [11]:
# combine gulf and atlantic corridors. For overlap areas use the maximum value
arcpy.management.MosaicToNewRaster(["GulfCorridor.tif", "AtlanticCorridor.tif"], OutWorkspace, maskFileName,"","","", 1,"MAXIMUM")

<Result 'D:/Blueprint/2023/corridors/MarineMammalMovement/working\\MarineMammalAndTurtleMovementAreas.tif'>